# 비모수 분석 종합 템플릿 (All-in-One)이 노트북은 `nonparametric_analysis` 패키지가 제공하는 **17종의 모든 분석 기능**을 예시와 함께 제공합니다.각 결과 아래에는 **통계 비전문가를 위한 '결과 해석 가이드'**가 포함되어 있습니다.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings

# Jupyter용 matplotlib 설정
%matplotlib inline

# 경고 무시
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

# Add src to path
sys.path.append(str(Path('../03_Code/src').resolve()))

import matplotlib.pyplot as plt
from nonparametric_analysis.analysis import nonparametric_methods as np_methods
from nonparametric_analysis.analysis import utils
from nonparametric_analysis.analysis.visualizations import setup_visualization

setup_visualization() # 한글 폰트 및 스타일 설정

def show_result(res):
    """분석 결과를 보기 좋게 출력하는 헬퍼 함수"""
    print("=" * 55)
    print("  분석 결과 요약")
    print("=" * 55)
    for k, v in res.items():
        if k == 'figure':
            continue
        if isinstance(v, pd.DataFrame):
            print(f"\n  [{k}]")
            display(v)
        elif isinstance(v, float):
            print(f"  {k:.<30s} {v:.4f}")
        else:
            print(f"  {k:.<30s} {v}")
    print("=" * 55)
    plt.show()

In [ ]:
# ============================================================
#   🔧 여기만 수정하세요! (분석 설정)
# ============================================================

# 0. 데이터 입력 방식 선택
#    방법 A: CSV 파일 → FILE_NAME에 파일명 입력 (기본)
#    방법 B: 복사/붙여넣기 → PASTE_DATA에 데이터 직접 붙여넣기
#           (DRM 문서 등 파일 저장이 어려운 경우 사용)
PASTE_DATA = """
"""
# ↑ 붙여넣기 사용법:
#   1. 위 쌍따옴표 사이의 빈 줄에 커서를 놓으세요
#   2. 원본 문서에서 데이터를 복사(Ctrl+C)하세요
#   3. 여기에 붙여넣기(Ctrl+V)하세요
#   4. 첫 행은 반드시 컬럼명이어야 합니다
#
# 예시 (쉼표 구분):
# PASTE_DATA = """
# 이름,매출액,지역
# 김철수,1520,서울
# 이영희,1680,부산
# """
#
# 예시 (탭 구분 - Excel에서 복사 시):
# PASTE_DATA = """
# 이름	매출액	지역
# 김철수	1520	서울
# """
# → 쉼표, 탭, 세미콜론 모두 자동 인식됩니다.

# 1. CSV 파일 이름 (PASTE_DATA가 비어있을 때만 사용됨)
#    02_Data/ 폴더에 CSV 파일을 넣으세요
FILE_NAME = "sample_nonparametric.csv"

# 2. 분석할 숫자 컬럼 (시계열/단일변수 분석 대상)
TARGET_COLUMN = "feature_1"

# 3. 두 그룹 비교 설정 (Mann-Whitney, K-S 검정용)
#    사용 안 하면 빈 문자열("")로 두세요
GROUP_COLUMN   = "group"        # 그룹을 구분하는 컬럼
GROUP_A_VALUE  = "control"      # A그룹 값
GROUP_B_VALUE  = "treatment"    # B그룹 값

# 4. 전/후 비교 설정 (Wilcoxon, 부호 검정용)
#    사용 안 하면 빈 문자열("")로 두세요
BEFORE_COLUMN  = ""             # 처리 전 측정값 컬럼명
AFTER_COLUMN   = ""             # 처리 후 측정값 컬럼명

# 5. 다중 독립 그룹 비교 (Kruskal-Wallis용, 3개 이상)
#    GROUP_COLUMN에 있는 값들을 리스트로 넣으세요
MULTI_GROUP_VALUES = []         # 예: ["서울", "부산", "대구"]

# 6. 반복 측정 비교 (Friedman용, 동일 대상의 여러 시점 측정)
#    각 시점이 별도 컬럼이어야 합니다
REPEATED_COLUMNS = []           # 예: ["1월_점수", "3월_점수", "6월_점수"]

# 7. 상관분석 컬럼 (비워두면 모든 숫자 컬럼 자동 사용)
CORRELATION_COLUMNS = []        # 예: ["매출액", "광고비", "방문수"]

# ============================================================
#   ⬇️ 아래부터는 수정하지 마세요!
# ============================================================

In [ ]:
# --- 데이터 로드, 검증, 준비 (수정 불필요) ---
import os
from io import StringIO

# 1) 데이터 로드 (붙여넣기 또는 CSV 파일)
if PASTE_DATA.strip():
    # 붙여넣기 데이터 사용
    df = pd.read_csv(StringIO(PASTE_DATA.strip()), sep=None, engine="python")
    _data_source = "붙여넣기"
    print("=" * 55)
    print("  📋 붙여넣기 데이터 로드 완료")
    print("=" * 55)
else:
    # CSV 파일 사용
    data_path = Path(f"../02_Data/{FILE_NAME}")
    if not data_path.exists():
        data_dir = Path("../02_Data")
        existing = [f for f in os.listdir(data_dir) if f.endswith(".csv")] if data_dir.exists() else []
        raise FileNotFoundError(
            f"\n❌ 파일을 찾을 수 없습니다: {FILE_NAME}\n"
            f"   02_Data/ 폴더에 CSV 파일을 넣었는지 확인하세요.\n"
            f"   현재 CSV 파일 목록: {existing}"
        )
    df = pd.read_csv(data_path)
    _data_source = f"CSV 파일 ({FILE_NAME})"
    print("=" * 55)
    print("  📂 CSV 파일 로드 완료")
    print("=" * 55)

print(f"  데이터 출처........... {_data_source}")
print(f"  행 수................ {len(df):,}개")
print(f"  열 수................ {len(df.columns)}개")
print()
print("  📋 컬럼 목록:")
for col in df.columns:
    dtype = "숫자" if df[col].dtype in ['int64', 'float64'] else "문자"
    null_cnt = df[col].isna().sum()
    null_str = f", 결측 {null_cnt}개" if null_cnt > 0 else ""
    print(f"    - {col} ({dtype}{null_str})")
print()

# 3) 설정값 검증
errors = []
if not TARGET_COLUMN:
    errors.append("TARGET_COLUMN이 비어있습니다. 분석할 숫자 컬럼을 지정하세요.")
elif TARGET_COLUMN not in df.columns:
    errors.append(f"TARGET_COLUMN '{TARGET_COLUMN}'이(가) 데이터에 없습니다. 가능한 컬럼: {list(df.columns)}")
elif not pd.api.types.is_numeric_dtype(df[TARGET_COLUMN]):
    errors.append(f"TARGET_COLUMN '{TARGET_COLUMN}'이(가) 숫자 컬럼이 아닙니다.")

if GROUP_COLUMN:
    if GROUP_COLUMN not in df.columns:
        errors.append(f"GROUP_COLUMN '{GROUP_COLUMN}'이(가) 데이터에 없습니다. 가능한 컬럼: {list(df.columns)}")
    else:
        vals = df[GROUP_COLUMN].unique()
        if GROUP_A_VALUE and GROUP_A_VALUE not in vals:
            errors.append(f"GROUP_A_VALUE '{GROUP_A_VALUE}'이(가) '{GROUP_COLUMN}' 컬럼에 없습니다. 가능한 값: {list(vals)}")
        if GROUP_B_VALUE and GROUP_B_VALUE not in vals:
            errors.append(f"GROUP_B_VALUE '{GROUP_B_VALUE}'이(가) '{GROUP_COLUMN}' 컬럼에 없습니다. 가능한 값: {list(vals)}")
        for mv in MULTI_GROUP_VALUES:
            if mv not in vals:
                errors.append(f"MULTI_GROUP_VALUES의 '{mv}'이(가) '{GROUP_COLUMN}' 컬럼에 없습니다. 가능한 값: {list(vals)}")

if BEFORE_COLUMN and BEFORE_COLUMN not in df.columns:
    errors.append(f"BEFORE_COLUMN '{BEFORE_COLUMN}'이(가) 데이터에 없습니다. 가능한 컬럼: {list(df.columns)}")
if AFTER_COLUMN and AFTER_COLUMN not in df.columns:
    errors.append(f"AFTER_COLUMN '{AFTER_COLUMN}'이(가) 데이터에 없습니다. 가능한 컬럼: {list(df.columns)}")

for rc in REPEATED_COLUMNS:
    if rc not in df.columns:
        errors.append(f"REPEATED_COLUMNS의 '{rc}'이(가) 데이터에 없습니다. 가능한 컬럼: {list(df.columns)}")

for cc in CORRELATION_COLUMNS:
    if cc not in df.columns:
        errors.append(f"CORRELATION_COLUMNS의 '{cc}'이(가) 데이터에 없습니다. 가능한 컬럼: {list(df.columns)}")

if errors:
    print("❌ 설정 오류가 있습니다:")
    for e in errors:
        print(f"   - {e}")
    print("\n  ⬆️ 위의 '설정 셀'을 수정한 후 다시 실행하세요.")
    raise ValueError("설정 오류 - 위의 메시지를 확인하세요.")
else:
    print("✅ 설정 검증 완료!")

# 4) 분석 데이터 준비
series = df[TARGET_COLUMN].dropna()

# 두 그룹 비교
if GROUP_COLUMN and GROUP_A_VALUE and GROUP_B_VALUE:
    group_a = df[df[GROUP_COLUMN] == GROUP_A_VALUE][TARGET_COLUMN].dropna()
    group_b = df[df[GROUP_COLUMN] == GROUP_B_VALUE][TARGET_COLUMN].dropna()
    HAS_TWO_GROUPS = len(group_a) > 0 and len(group_b) > 0
else:
    group_a, group_b = pd.Series(dtype=float), pd.Series(dtype=float)
    HAS_TWO_GROUPS = False

# 전/후 비교
if BEFORE_COLUMN and AFTER_COLUMN:
    before = df[BEFORE_COLUMN].dropna().values
    after = df[AFTER_COLUMN].dropna().values
    min_len = min(len(before), len(after))
    before, after = before[:min_len], after[:min_len]
    HAS_PAIRED = min_len > 0
else:
    before, after = None, None
    HAS_PAIRED = False

# 다중 독립 그룹
if GROUP_COLUMN and len(MULTI_GROUP_VALUES) >= 3:
    multi_groups = [
        df[df[GROUP_COLUMN] == v][TARGET_COLUMN].dropna().values
        for v in MULTI_GROUP_VALUES
    ]
    HAS_MULTI_GROUPS = all(len(g) > 0 for g in multi_groups)
else:
    multi_groups = []
    HAS_MULTI_GROUPS = False

# 반복 측정
if len(REPEATED_COLUMNS) >= 3:
    repeated_data = [df[col].dropna().values for col in REPEATED_COLUMNS]
    min_repeated = min(len(d) for d in repeated_data)
    repeated_data = [d[:min_repeated] for d in repeated_data]
    HAS_REPEATED = min_repeated > 0
else:
    repeated_data = []
    HAS_REPEATED = False

# 상관분석
if CORRELATION_COLUMNS:
    corr_df = df[CORRELATION_COLUMNS].dropna()
else:
    corr_df = df.select_dtypes(include=[np.number]).dropna()

# 5) 준비 결과 요약
print()
print("=" * 55)
print("  📊 분석 데이터 준비 완료")
print("=" * 55)
print(f"  단일변수 (series)........ {len(series)}개")
if HAS_TWO_GROUPS:
    print(f"  두 그룹 비교............ ✅ {GROUP_A_VALUE}({len(group_a)}개) vs {GROUP_B_VALUE}({len(group_b)}개)")
else:
    print(f"  두 그룹 비교............ ⏭️ 건너뜀 (미설정)")
if HAS_PAIRED:
    print(f"  전/후 비교.............. ✅ {min_len}쌍")
else:
    print(f"  전/후 비교.............. ⏭️ 건너뜀 (미설정)")
if HAS_MULTI_GROUPS:
    print(f"  다중 그룹 비교.......... ✅ {len(MULTI_GROUP_VALUES)}개 그룹")
else:
    print(f"  다중 그룹 비교.......... ⏭️ 건너뜀 (미설정)")
if HAS_REPEATED:
    print(f"  반복 측정 비교.......... ✅ {len(REPEATED_COLUMNS)}시점 x {min_repeated}개")
else:
    print(f"  반복 측정 비교.......... ⏭️ 건너뜀 (미설정)")
print(f"  상관분석 컬럼........... {len(corr_df.columns)}개")
print("=" * 55)

print("\n📊 데이터 미리보기 (처음 5행):")
display(df.head())

#### 1.1 정규성 검정 (Normality Test)


In [ ]:
res = np_methods.test_normality(series, name=TARGET_COLUMN)
show_result(res)

#### 1.2 런 검정 (Runs Test - 무작위성)


In [ ]:
res = np_methods.runs_test_analysis(series, name=TARGET_COLUMN)
show_result(res)

#### 1.3 추세 분석 (Mann-Kendall Trend)


In [ ]:
res = np_methods.mann_kendall_test(series, name=TARGET_COLUMN)
show_result(res)

#### 1.4 변곡점 탐지 (Pettitt Test)


In [ ]:
res = np_methods.pettitt_test(series, name=TARGET_COLUMN)
show_result(res)

#### 1.5 다중 구간 분할 (PELT)


In [ ]:
res = np_methods.detect_changepoints_pelt(series, name=TARGET_COLUMN)
show_result(res)

##### 💡 결과 해석

**🎯 이 분석의 목적**: 데이터의 패턴이 바뀌는 **여러 지점을 동시에** 모두 찾습니다.

**📊 결과 읽는 법**:
- **Change Points = [10, 35, 78]** → 3개의 변화 지점 발견
  - → 데이터가 4개 구간으로 나뉨 (1~10, 11~35, 36~78, 79~끝)
  - → 각 구간마다 다른 특성(평균, 분산 등)을 가짐
- **Change Points = []** → 변화 지점 없음
  - → 데이터가 처음부터 끝까지 일관된 패턴 유지

**💼 실무 활용**:
- 예: 시간대별 서버 부하 → Change Points = [8, 12, 18, 22]
  - 판단: "출근(8시), 점심(12시), 퇴근(18시), 심야(22시)에 패턴 변화. 각 시간대별 리소스 할당 필요"
- 예: 분기별 매출 → Change Points = [90, 180]
  - 판단: "90일, 180일차에 매출 패턴 변화. 해당 시점 프로모션/계절성 분석"
- 예: 제품 품질 → Change Points = [150]
  - 판단: "150번째 제품부터 품질 변화. 공정 변경 또는 원자재 변경 시점 확인"

**📈 차트 확인**:
- **빨간 수직선들**: 각 변화 지점 표시
- **구간별 색상**: 다른 색상으로 구분된 구간들
- **각 구간의 평균선**: 구간마다 다른 수준

#### 2.1 두 독립 그룹 비교 (Mann-Whitney U)


In [ ]:
if HAS_TWO_GROUPS:
    res = np_methods.mann_whitney_test(group_a, group_b, name1=GROUP_A_VALUE, name2=GROUP_B_VALUE)
    show_result(res)
else:
    print("⏭️ 두 그룹 비교를 건너뜁니다.")
    print("   → 설정 셀에서 GROUP_COLUMN, GROUP_A_VALUE, GROUP_B_VALUE를 지정하면 실행됩니다.")

#### 2.2 두 분포 비교 (K-S Test)


In [ ]:
if HAS_TWO_GROUPS:
    res = np_methods.ks_test(group_a, group_b, name1=GROUP_A_VALUE, name2=GROUP_B_VALUE)
    show_result(res)
else:
    print("⏭️ 두 분포 비교를 건너뜁니다.")
    print("   → 설정 셀에서 GROUP_COLUMN, GROUP_A_VALUE, GROUP_B_VALUE를 지정하면 실행됩니다.")

#### 2.3 짝지어진 그룹 비교 (Wilcoxon Signed Rank)


In [ ]:
if HAS_PAIRED:
    res = np_methods.wilcoxon_paired_test(before, after, name=f"{BEFORE_COLUMN} vs {AFTER_COLUMN}")
    show_result(res)
else:
    print("⏭️ 전/후 비교를 건너뜁니다.")
    print("   → 설정 셀에서 BEFORE_COLUMN, AFTER_COLUMN을 지정하면 실행됩니다.")
    print("   (예: 동일 대상의 치료 전/후, 교육 전/후 측정값)")

#### 2.4 부호 검정 (Sign Test)


In [ ]:
if HAS_PAIRED:
    res = np_methods.sign_test(before, after)
    show_result(res)
else:
    print("⏭️ 부호 검정을 건너뜁니다.")
    print("   → 설정 셀에서 BEFORE_COLUMN, AFTER_COLUMN을 지정하면 실행됩니다.")

#### 2.5 세 독립 그룹 비교 (Kruskal-Wallis)


In [ ]:
if HAS_MULTI_GROUPS:
    res = np_methods.kruskal_wallis_test(multi_groups, group_names=MULTI_GROUP_VALUES)
    show_result(res)
else:
    print("⏭️ 다중 그룹 비교를 건너뜁니다.")
    print("   → 설정 셀에서 MULTI_GROUP_VALUES에 3개 이상의 그룹값을 지정하면 실행됩니다.")
    print("   (예: MULTI_GROUP_VALUES = [\"서울\", \"부산\", \"대구\"])")

#### 2.6 반복 측정 비교 (Friedman Test)


In [ ]:
if HAS_REPEATED:
    res = np_methods.friedman_test(repeated_data, condition_names=REPEATED_COLUMNS)
    show_result(res)
else:
    print("⏭️ 반복 측정 비교를 건너뜁니다.")
    print("   → 설정 셀에서 REPEATED_COLUMNS에 3개 이상의 시점 컬럼을 지정하면 실행됩니다.")
    print("   (예: REPEATED_COLUMNS = [\"1월_점수\", \"3월_점수\", \"6월_점수\"])")
    print("   ※ 동일 대상을 여러 시점에 측정한 데이터여야 합니다.")

##### 💡 결과 해석

**🎯 이 분석의 목적**: 3개 이상의 시점/조건에서 **동일한 대상**을 반복 측정했을 때 차이를 확인합니다.

**📊 결과 읽는 법**:
- **p < 0.05** → 시점/조건 간에 **차이가 존재**합니다
  - → 적어도 한 시점이 다른 시점들과 다름
  - → 시간에 따른 변화 또는 조건 효과가 있음
- **p ≥ 0.05** → 시점/조건 간에 **차이가 없습니다**
  - → 시간이 지나도 변화 없음
  - → 조건을 바꿔도 효과 없음

**💬 사용 조건**:
- **반드시 동일 대상**의 반복 측정이어야 함
- 예: 같은 환자를 1개월, 3개월, 6개월 후 측정

**💼 실무 활용**:
- 예: 다이어트 프로그램 (시작/1개월/3개월/6개월) → p = 0.001
  - 판단: "시간에 따라 체중 변화 있음. 사후검정으로 어느 시점에 가장 효과적인지 파악"
- 예: 약물 용량 (저용량/중용량/고용량) 동일 환자 → p = 0.03
  - 판단: "용량에 따라 효과 차이 있음. 최적 용량 찾기 위한 추가 분석 필요"
- 예: 계절별(봄/여름/가을/겨울) 동일 매장 매출 → p = 0.67
  - 판단: "계절 영향 없음. 연중 일관된 마케팅 전략 유지"

**📈 차트 확인**:
- **시점별 Box Plot**: 시점에 따라 상자 위치가 바뀌는지 확인
- **평균 순위**: 어느 시점의 순위가 가장 높은지/낮은지
- **변화 추세선**: 시간 흐름에 따른 패턴 확인

#### 3.1 상관 행렬 (Heatmap)


In [ ]:
if len(corr_df.columns) >= 2:
    res = np_methods.correlation_matrix_nonparametric(corr_df)
    show_result(res)
else:
    print("⏭️ 상관 행렬 분석을 건너뜁니다.")
    print("   → 숫자 컬럼이 2개 이상 필요합니다.")

#### 3.2 다양한 상관 분석 (Kendall, Distance)


In [ ]:
num_cols = corr_df.columns.tolist()
if len(num_cols) >= 2:
    x = corr_df[num_cols[0]]
    y = corr_df[num_cols[1]]
    print(f"📊 분석 대상: {num_cols[0]} vs {num_cols[1]}")
    print()

    res_k = np_methods.kendall_corr(x, y)
    print(f"Kendall Tau: {res_k['correlation']:.4f}, p-value: {res_k['p_value']:.4f}")
    plt.show()

    res_d = np_methods.distance_correlation(x, y)
    print(f"Distance Corr: {res_d['dcor']:.4f}, p-value: {res_d['p_value']:.4f}")
    plt.show()
else:
    print("⏭️ 개별 상관 분석을 건너뜁니다.")
    print("   → 숫자 컬럼이 2개 이상 필요합니다.")

##### 💡 결과 해석

**📊 Kendall Tau (켄달 타우)**:
- **목적**: 순위 동점이 많을 때 더 정확한 상관 분석
- **값 해석**:
  - **+1에 가까움**: 강한 양의 관계
  - **0에 가까움**: 관계 없음
  - **-1에 가까움**: 강한 음의 관계
- **Spearman과의 차이**: 동일 값(동점)이 많을 때 더 안정적

**📊 Distance Correlation (거리 상관)**:
- **목적**: **곡선 관계(비선형)**도 찾아냅니다
- **값 해석**:
  - **0**: 완전히 독립적 (관계 없음)
  - **1**: 완전히 종속적 (강한 관계)
  - 중요: 항상 0~1 사이 값 (음수 없음)
- **일반 상관과의 차이**: 
  - 일반 상관: 직선 관계만 찾음 (y = ax + b)
  - Distance: 곡선 관계도 찾음 (y = x², y = sin(x) 등)

**💼 실무 활용**:
- 예: Kendall Tau = 0.75 vs Spearman = 0.68 (동점 많은 데이터)
  - 판단: "Kendall이 더 신뢰할 만함. Spearman은 동점에 취약"
- 예: 일반 상관 = 0.1, Distance 상관 = 0.8
  - 판단: "직선 관계는 약하지만 곡선 관계는 강함. 비선형 모델 고려 필요"
- 예: Distance 상관 = 0.05
  - 판단: "직선이든 곡선이든 관계 없음. 두 변수 독립적"

#### 4.1 부트스트랩 신뢰구간 (Bootstrap CI)


In [ ]:
res = np_methods.bootstrap_ci(series, stat_func=np.mean, n_boot=1000, name=TARGET_COLUMN)
show_result(res)

#### 4.2 순열 검정 (Permutation Test)


In [ ]:
if HAS_TWO_GROUPS:
    res = np_methods.permutation_test(
        group_a, group_b,
        stat_func=np.mean, n_perm=1000,
        name1=GROUP_A_VALUE, name2=GROUP_B_VALUE,
    )
    show_result(res)
else:
    print("⏭️ 순열 검정을 건너뜁니다.")
    print("   → 설정 셀에서 GROUP_COLUMN, GROUP_A_VALUE, GROUP_B_VALUE를 지정하면 실행됩니다.")

##### 💡 결과 해석

**🎯 이 분석의 목적**: 정규분포 가정이 불가능할 때, **두 그룹의 차이가 우연인지** 검정합니다.

**📊 결과 읽는 법**:
- **p < 0.05** → 두 그룹 차이가 **진짜**입니다
  - → 우연으로 이런 차이가 나올 확률 5% 미만
  - → 그룹 A와 B는 실제로 다른 모집단
- **p ≥ 0.05** → 두 그룹 차이가 **우연**입니다
  - → 운이 나빠서 차이가 난 것일 수 있음
  - → 두 그룹을 다르다고 결론 내릴 수 없음

**💬 순열검정이란?**:
- 데이터를 **무작위로 섞어서** 관찰된 차이가 우연인지 확인
- 예: 그룹 라벨을 1000번 섞어보고, 원래 차이보다 큰 차이가 몇 번 나오는지 계산
- 정규분포, 동분산 등 **어떤 가정도 필요 없음**

**💼 실무 활용**:
- 예: 신약 vs 위약 평균 차이 = 5, p = 0.002
  - 판단: "차이가 우연이 아님. 신약이 진짜 효과 있음. 임상 시험 성공"
- 예: A/B 테스트 전환율 차이 = 2%, p = 0.35
  - 판단: "2% 차이는 우연일 수 있음. A와 B 중 어느 것이 좋다고 결론 못 내림. 샘플 더 모으거나 다른 방법 시도"
- 예: 시간대별 매출 차이 = 100만원, p = 0.01
  - 판단: "시간대별 차이가 진짜임. 피크 시간대 인력 배치 강화 필요"

**📈 차트 확인**:
- **순열 분포 히스토그램**: 우연히 나올 수 있는 차이들의 분포
- **관찰된 차이 (빨간 선)**: 실제로 관찰된 차이
- **빨간 선이 히스토그램 끝에 위치**: 매우 드문 경우 → 우연 아님 → p 값 작음